In [30]:
import cv2
import numpy as np
import pandas as pd
import os
import glob

In [20]:
# 체커보드의 차원 정의
CHECKERBOARD = (6,9) # 체커보드 행과 열당 내부 코너 수
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# 각 체커보드 이미지에 대한 3D 점 벡터를 저장할 벡터 생성
objpoints = []
# 각 체커보드 이미지에 대한 2D 점 벡터를 저장할 벡터 생성
imgpoints = [] 
# 3D 점의 세계 좌표 정의
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None

In [21]:
# 주어진 디렉터리에 저장된 개별 이미지의 경로 추출
images = glob.glob('./Calibration_dataset/*.jpg')

# 체커보드 2D 좌표 찾기 

### 1. 체커보드 찾고 코너 좌표 반환
```retval, corners = cv2.findChessboardCorners(image, patternSize, flags)```
* patternSize : 체커보드 행과 열당 내부 코너 수
* corners : 감지된 코너의 출력 배열
* flags : 다양한 작업 플래그

### 2. 체커보드 코너 개선
좋은 결과를 얻으려면 sub-pixel 수준의 정확도로 코너 위치를 얻는 것이 중요하다.      
원본 이미지와 코너 위치를 가져와서 원래 위치의 작은 이웃 내에서 가장 좋은 코너 위치를 찾는다.       
종료 기준을 지정해야 한다. 
```cv2.cornerSubPix(image, corners, winSize, zeroZone, criteria)```
* corners : 입력 코너의 초기 좌표와 출력을 위해 제공되는 개선 좌표
* winSize : 검색 창의 측면 길이의 절반
* zeroZone : 아래 공식의 합산이 이루어지지 않은 탐색 영역 중앙에 있는 사각 영역(dead region) 크기의 절반. 자기상관 행렬(autocorrelation matrix)의 가능한 특이성을 피하기 위해 때때로 사용. (-1,-1) 값은 그러한 크기가 없음을 나타냄.
* criteria : 코너 미세 조정(corner refinement)의 반복 프로세스 종료 기준. 즉, 코너 위치 미세 조정 프로세스는 criteria.maxCount 반복 후 또는 일부 반복에서 코너 위치가 criteria.epsilon보다 작게 이동할 때  중지됨.

In [22]:
for fname in images:
    img = cv2.imread(fname)
    # 그레이 스케일로 변환
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # 체커보드 코너 찾기
    # 이미지에서 원하는 개수의 코너가 발견되면 ret = true
    ret, corners = cv2.findChessboardCorners(gray,
                                             CHECKERBOARD,
                                             cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
    # 원하는 개수의 코너가 감지되면,
    # 픽셀 좌표 미세조정 -> 체커보드 이미지 표시
    if ret == True:
        objpoints.append(objp)
        # 주어진 2D 점에 대한 픽셀 좌표 미세조정
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
        imgpoints.append(corners2)
        # 코너 그리기 및 표시
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
    cv2.imshow('img',img)
    cv2.waitKey(0)
cv2.destroyAllWindows()
h,w = img.shape[:2] # 480, 640

# 카메라 캘리브레이션

```retval, cameraMatrix, distCoeffs, rvecs, tvecs = cv2.calibrateCamera(objectPoints, imagePoints, imageSize)```
* objectPoints : 3D 점 벡터로 구성된 벡터. 외부 벡터는 패턴 사진의 수만큼 요소를 포함
* imagePoints : 2D 이미지 점 벡터로 구성된 벡터
* imageSize	: 이미지의 크기
* cameraMatrix : 내부 카메라 행렬
* distCoeffs : 렌즈 왜곡 계수(Lens distortion coefficients)
* rvecs : 회전은 3×1 벡터로 지정. 벡터의 방향은 회전 축을 지정하고 벡터의 크기는 회전 각을 지정
* tvecs	: 3×1 이동 벡터

In [23]:
# 알려진 3D 점(objpoints) 값과 감지된 코너의 해당 픽셀 좌표(imgpoints) 전달, 카메라 캘리브레이션 수행
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

내부 카메라 행렬

In [24]:
print("Camera matrix : \n") # 내부 카메라 행렬
print(mtx)

Camera matrix : 

[[1.13365105e+03 0.00000000e+00 6.50904750e+02]
 [0.00000000e+00 1.13139467e+03 3.92421490e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


렌즈 왜곡 계수(Lens distortion coefficients)

In [25]:
print("dist : \n") # 렌즈 왜곡 계수(Lens distortion coefficients)
print(dist)

dist : 

[[-0.22067338 -0.37452586 -0.00147645  0.00081546  0.71218214]]


Undistort

In [29]:
# undistort
img=cv2.imread('./Calibration_dataset/calibration6.jpg')
h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibresult.png', dst)

True

In [52]:
import csv
dist_data = list()
f = open("./Const/dst.csv",'r')
rea = csv.reader(f)
for row in rea:
    dist_data.extend(row)
f.close

converted_list = [float(num) for num in dist_data]
dist_data = np.array([dist_data], dtype=float)
print(dist_data)

[[-0.02010607 -0.01730982 -0.03868419  0.00366522 -0.03838312  0.02813846
   0.02444793 -0.06326856  0.          0.          0.          0.
   0.          0.        ]]


In [44]:
mtx_data = list()
f = open("./Const/mtx.csv",'r')
r = csv.reader(f)
for row in r:
    mtx_data.append(row)
f.close

mtx_data = np.array(mtx_data, dtype=float)
print(mtx_data)

[[9.31567222e+02 0.00000000e+00 8.73384926e+02]
 [0.00000000e+00 9.31567222e+02 1.07022471e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [48]:
i=1
while True:
    img=cv2.imread('./dataset/calib_{0}.jpg'.format(i))
    h, w = img.shape[:2]
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx_data, dist_data, (w,h), 1, (w,h))

    dst = cv2.undistort(img, mtx_data, dist_data, None, newcameramtx)

    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]
    cv2.imwrite('calibresult_{0}.png'.format(i), dst)
    i=i+1
    if i==21:
        break

회전 벡터

In [26]:
print("rvecs : \n") # 회전 벡터
print(rvecs)

rvecs : 

(array([[ 0.56058364],
       [ 0.24738196],
       [-1.50327664]]), array([[-0.31975595],
       [ 0.38529646],
       [-1.53356819]]), array([[-0.48397354],
       [ 0.54529693],
       [-1.49734872]]), array([[-0.82635033],
       [ 0.28476593],
       [-1.89636445]]), array([[ 0.43142097],
       [-0.3260521 ],
       [-1.54367237]]), array([[ 0.33700114],
       [-0.37064783],
       [-1.52400195]]), array([[ 0.38028665],
       [-0.29694257],
       [-1.60265958]]), array([[ 0.21675552],
       [ 0.13430721],
       [-1.55324044]]), array([[ 0.17798513],
       [ 0.10810855],
       [-1.56815434]]), array([[-0.21785387],
       [ 0.37089327],
       [-1.49158263]]), array([[-0.31248266],
       [ 0.27738728],
       [-1.55359268]]), array([[-0.00578542],
       [ 0.03630805],
       [-1.57639148]]), array([[-0.48554946],
       [ 0.52418942],
       [-1.49992988]]), array([[ 0.56276053],
       [-0.51118997],
       [-1.52388076]]), array([[ 0.42597766],
       [-0.7152

이동 벡터

In [27]:
print("tvecs : \n") # 이동 벡터
print(tvecs)

tvecs : 

(array([[-2.17462619],
       [ 3.52325315],
       [21.66110903]]), array([[-16.54363737],
       [  1.3323944 ],
       [ 31.56328052]]), array([[ 0.14106533],
       [ 1.41096513],
       [21.72801862]]), array([[-4.35412705],
       [ 2.64368223],
       [24.21311643]]), array([[ 5.65327771],
       [ 0.43557922],
       [20.61281979]]), array([[ 4.7258506 ],
       [ 3.42605523],
       [19.41203179]]), array([[ 5.42204832],
       [-0.15924217],
       [19.59122403]]), array([[-3.78594564],
       [ 3.46865369],
       [17.81514657]]), array([[-3.36467837],
       [ 0.69972342],
       [18.36604614]]), array([[-12.87771783],
       [ -0.75524109],
       [ 23.72894757]]), array([[-13.09954128],
       [  4.39624418],
       [ 23.92419602]]), array([[-4.45439465],
       [ 0.98562857],
       [30.00744945]]), array([[-9.16890211],
       [ 1.54794605],
       [31.7784304 ]]), array([[ 1.06249134],
       [ 1.98150347],
       [19.40574887]]), array([[-0.74024701],
      